In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import timedelta
import time
import datetime
import os

# Préparation des inputs

In [20]:
# Paramètre : Regroupement des données pour les input
step_group = "2min"

# Paramètre : Intervalle de temps pour une ligne d'entrainement
delta_train = "2h"

# Paramètre : intervalle de temps sur lequel on souhaite faire une prédiction à une date donnée
delta_prediction = "30min"

# Paramètre : temps de window slide pour chaque nouvelle inpute
window_slide = "2h"

Chargement des données

In [10]:
data = pd.read_csv('/Users/gauthier/Desktop/data_processed')

Passage des dates au format Timestamp

In [11]:
data['date'] = data['date'].apply(lambda x : pd.Timestamp(x))

One hot encoding pour la variable CBN

In [12]:
data['CBN'] = pd.Categorical(data['CBN'])

# Créer les one hot encodings pour chaque variabble
dfDummies_cbn = pd.get_dummies(data['CBN'], prefix = 'CBN')

# Concatener les one hot encodings avec les données
data = pd.concat([data, dfDummies_cbn], axis=1)

# Suppression de la variable CBBN
data.drop(columns=['CBN'], inplace=True)

# On ordonne les data par "date"
data.sort_values(by='date', inplace=True)

# Réindexage des lignes
data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)

In [13]:
data.head()

,date,CBN_0 02,CBN_0 03,CBN_0 04,CBN_0 05,CBN_0 06,CBN_0 07,CBN_0 08,CBN_0 09,CBN_0 10,...,CBN_X 06,CBN_X 07,CBN_X 08,CBN_X 09,CBN_X 10,CBN_X 11,CBN_X 12,CBN_Y,CBN_Z,CBN_j 04
0,2018-07-21 04:52:46,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-07-21 04:52:49,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-07-21 04:52:57,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018-07-21 04:53:01,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-07-21 04:53:04,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Formatage des input

In [14]:
start = time.time()

grouped_data = data.groupby(pd.Grouper(key="date", freq=step_group)).sum()

print("Temps de création : {:.2f} s".format(time.time() - start))

# Ajout d'un index
grouped_data.reset_index(inplace=True)

Temps de création : 37.71 s


Ajout variable mois et jours de la semaine

In [15]:
grouped_data['month'] = grouped_data['date'].apply(lambda x : x.month)
grouped_data['weekday'] = grouped_data['date'].apply(lambda x : x.weekday())

One hot encoding pour les varialbes jours de la semaine et mois

In [16]:
grouped_data['month'] = pd.Categorical(grouped_data['month'])
grouped_data['weekday'] = pd.Categorical(grouped_data['weekday'])

# Créer les one hot encodings pour chaque variabble
dfDummies_month = pd.get_dummies(grouped_data['month'], prefix = 'month')
dfDummies_weekday = pd.get_dummies(grouped_data['weekday'], prefix = 'weekday')

# Concatener les one hot encodings avec les données
grouped_data = pd.concat([grouped_data, dfDummies_weekday, dfDummies_month], axis=1)

# Suppression de la variable mois et jour de la semaine
grouped_data.drop(columns=['month', 'weekday'], inplace=True)

In [17]:
grouped_data.head()

,date,CBN_0 02,CBN_0 03,CBN_0 04,CBN_0 05,CBN_0 06,CBN_0 07,CBN_0 08,CBN_0 09,CBN_0 10,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2018-07-21 04:52:00,0,0,0,17,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2018-07-21 04:54:00,0,0,0,14,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2018-07-21 04:56:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,2018-07-21 04:58:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2018-07-21 05:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Indices pour sélection les lignes

In [21]:
index_delta_train = pd.Timedelta(delta_train).seconds // pd.Timedelta(step_group).seconds
index_delta_prediction = pd.Timedelta(delta_prediction).seconds // pd.Timedelta(step_group).seconds
index_window_slide = pd.Timedelta(window_slide).seconds // pd.Timedelta(step_group).seconds

Initialisation des listes vide

In [22]:
X = list()
y = list()

Ajout des variables d'entrainement dans ces listes

In [23]:
start = time.time()

index = 0

while (index + index_delta_train < grouped_data.shape[0]) and (index+index_delta_prediction < grouped_data.shape[0]):
    index_end = index+index_delta_train
    X.append(list(grouped_data.iloc[index:index_end].values))
    y.append(list(grouped_data.iloc[index_end:index_end+index_delta_prediction, 1:-(7+12)].sum().values))
    index += index_window_slide
    
print("Temps d'exécution : {:.2} s".format(time.time()-start))

Temps d'exécution : 5.0 s


On passe les variables au format numpy array

In [24]:
X = np.array(X)
y = np.array(y)

Séparation des données entre le training set (90%) et le test set (10%)

In [34]:
# Test set
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

date_test = X_test[:,:,0].flatten()
X_test = np.delete(X_test, 0, axis=2)


# Train and dev set
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, random_state=1)

date_train = X_train[:,:,0].flatten()
X_train = np.delete(X_train, 0, axis=2)

date_val = X_val[:,:,0].flatten()
X_val = np.delete(X_val, 0, axis=2)

Sauvegarde des données

In [35]:
out_dir_path = "inputs/group{}_dtrain{}_dpred{}_window{}".format(step_group, delta_train, delta_prediction, window_slide)

# Si le dossier n'existe pas on le crée
if not os.path.isdir(out_dir_path):
    os.mkdir(out_dir_path)
    
# Sauvegarde des tableaux numpy
np.save(out_dir_path+"/X_train", X_train)
np.save(out_dir_path+"/y_train", y_train)
np.save(out_dir_path+"/date_train", date_train)
np.save(out_dir_path+"/X_test", X_test)
np.save(out_dir_path+"/y_test", y_test)
np.save(out_dir_path+"/date_test", date_test)
np.save(out_dir_path+"/X_val", X_val)
np.save(out_dir_path+"/y_val", y_val)
np.save(out_dir_path+"/date_val", date_val)